In [1]:
import random
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.backend import reshape
from keras.utils.np_utils import to_categorical
import os

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Get an empty board
#
# 0 indicates an empty space, 1 indicates an 'X' (player 1), and 2 indicates an 'O' (player 2)
#
# Initially the board is empty, so we return a 3x3 array of zeroes.
def initBoard():
    board = [
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]
    ]
    return board

# Print the current state of the board
def printBoard(board):
    for i in range(len(board)):
        for j in range(len(board[i])):
            mark = ' '
            if board[i][j] == 1:
                mark = 'X'
            elif board[i][j] == 2:
                mark = 'O'
            if (j == len(board[i]) - 1):
                print(mark)
            else:
                print(str(mark) + "|", end='')
        if (i < len(board) - 1):
            print("-----")

In [3]:
# Get a list of valid moves (indices into the board)
def getMoves(board):
    moves = []
    for i in range(len(board)):
        for j in range(len(board[i])):
            if board[i][j] == 0:
                moves.append((i, j))
    return moves

# Declare a winner
#
# -1 = game not over
#  0 = draw
#  1 = 'X' wins (player 1)
#  2 = 'O' wins (player 2)
def getWinner(board):
    candidate = 0
    won = 0
    
    # Check rows
    for i in range(len(board)):
        candidate = 0
        for j in range(len(board[i])):
            
            # Make sure there are no gaps
            if board[i][j] == 0:
                break
            
            # Identify the front-runner
            if candidate == 0:
                candidate = board[i][j]
            
            # Determine whether the front-runner has all the slots
            if candidate != board[i][j]:
                break
            elif j == len(board[i]) - 1:
                won = candidate
    
    if won > 0:
        return won

    # Check columns
    for j in range(len(board[0])):
        candidate = 0
        for i in range(len(board)):
            
            # Make sure there are no gaps
            if board[i][j] == 0:
                break
            
            # Identify the front-runner
            if candidate == 0:
                candidate = board[i][j]
            
            # Determine whether the front-runner has all the slots
            if candidate != board[i][j]:
                break
            elif i == len(board) - 1:
                won = candidate
    
    if won > 0:
        return won
    
    # Check diagonals
    candidate = 0
    for i in range(len(board)):
        if board[i][i] == 0:
            break
        if candidate == 0:
            candidate = board[i][i]
        if candidate != board[i][i]:
            break
        elif i == len(board) - 1:
            won = candidate
    
    if won > 0:
        return won
    
    candidate = 0
    for i in range(len(board)):
        if board[2 - i][2 - i] == 0:
            break
        if candidate == 0:
            candidate = board[2 - i][2 - i]
        if candidate != board[2 - i][2 - i]:
            break
        elif i == len(board) - 1:
            won = candidate
    
    if won > 0:
        return won
    
    # Still no winner?
    if (len(getMoves(board)) == 0):
        # It's a draw
        return 0
    else:
        # Still more moves to make
        return -1

In [4]:
# Test helper methods
b = initBoard()
printBoard(b)
print(getWinner(b))
print(getMoves(b))

b[0][0] = 1
b[1][1] = 1
b[2][2] = 1
printBoard(b)
print(getWinner(b))

b[0][2] = 2
b[1][2] = 2
b[2][2] = 2
printBoard(b)
print(getWinner(b))

b[0][1] = 1
b[1][0] = 2
b[2][0] = 1
b[2][1] = 2
b[2][2] = 1
b[0][0] = 2
printBoard(b)
print(getWinner(b))

 | | 
-----
 | | 
-----
 | | 
-1
[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]
X| | 
-----
 |X| 
-----
 | |X
1
X| |O
-----
 |X|O
-----
 | |O
2
O|X|O
-----
O|X|O
-----
X|O|X
0


In [5]:
random.seed()

# Get best next move for the given player at the given board position
def bestMove(board, model, player, rnd=0):
    scores = []
    moves = getMoves(board)
    
    # Make predictions for each possible move
    for i in range(len(moves)):
        future = np.array(board)
        future[moves[i][0]][moves[i][1]] = player
        prediction = model.predict(future.reshape((-1, 9)))[0]
        if player == 1:
            winPrediction = prediction[1]
            lossPrediction = prediction[2]
        else:
            winPrediction = prediction[2]
            lossPrediction = prediction[1]
        drawPrediction = prediction[0]
        if winPrediction - lossPrediction > 0:
            scores.append(winPrediction - lossPrediction)
        else:
            scores.append(drawPrediction - lossPrediction)

    # Choose the best move with a random factor
    bestMoves = np.flip(np.argsort(scores))
    for i in range(len(bestMoves)):
        if random.random() * rnd < 0.5:
            return moves[bestMoves[i]]

    # Choose a move completely at random
    return moves[random.randint(0, len(moves) - 1)]


In [6]:
# Simulate a game
def simulateGame(p1=None, p2=None, rnd=0):
    history = []
    board = initBoard()
    playerToMove = 1
    
    while getWinner(board) == -1:
        
        # Chose a move (random or use a player model if provided)
        move = None
        if playerToMove == 1 and p1 != None:
            move = bestMove(board, p1, playerToMove, rnd)
        elif playerToMove == 2 and p2 != None:
            move = bestMove(board, p2, playerToMove, rnd)
        else:
            moves = getMoves(board)
            move = moves[random.randint(0, len(moves) - 1)]
        
        # Make the move
        board[move[0]][move[1]] = playerToMove
        
        # Add the move to the history
        history.append((playerToMove, move))
        
        # Switch the active player
        playerToMove = 1 if playerToMove == 2 else 2
        
    return history

# Simulate a game
history = simulateGame()
print(history)

[(1, (0, 2)), (2, (0, 1)), (1, (1, 0)), (2, (1, 2)), (1, (2, 0)), (2, (2, 2)), (1, (1, 1)), (2, (2, 1)), (1, (0, 0))]


In [7]:
# Reconstruct the board from the move list
def movesToBoard(moves):
    board = initBoard()
    for move in moves:
        player = move[0]
        coords = move[1]
        board[coords[0]][coords[1]] = player
    return board

board = movesToBoard(history)
printBoard(board)
print(getWinner(board))

X|O|X
-----
X|X|O
-----
X|O|O
1


In [8]:
games = [simulateGame() for _ in range(10000)]

In [9]:
# Aggregate win/loss/draw stats for a player
def gameStats(games, player=1):
    stats = {"win": 0, "loss": 0, "draw": 0}
    for game in games:
        result = getWinner(movesToBoard(game))
        if result == -1:
            continue
        elif result == player:
            stats["win"] += 1
        elif result == 0:
            stats["draw"] += 1
        else:
            stats["loss"] += 1
    
    winPct = stats["win"] / len(games) * 100
    lossPct = stats["loss"] / len(games) * 100
    drawPct = stats["draw"] / len(games) * 100

    print("Results for player %d:" % (player))
    print("Wins: %d (%.1f%%)" % (stats["win"], winPct))
    print("Loss: %d (%.1f%%)" % (stats["loss"], lossPct))
    print("Draw: %d (%.1f%%)" % (stats["draw"], drawPct))

gameStats(games)
print()
gameStats(games, player=2)

Results for player 1:
Wins: 5230 (52.3%)
Loss: 2444 (24.4%)
Draw: 2326 (23.3%)

Results for player 2:
Wins: 2444 (24.4%)
Loss: 5230 (52.3%)
Draw: 2326 (23.3%)


In [10]:
def getModel():
    numCells = 9 # How many cells in a 3x3 tic-tac-toe board?
    outcomes = 3 # How many outcomes are there in a game? (draw, X-wins, O-wins)
    model = Sequential()
    model.add(Dense(200, activation='relu', input_shape=(9, )))
    model.add(Dropout(0.2))
    model.add(Dense(125, activation='relu'))
    model.add(Dense(75, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(25, activation='relu'))
    model.add(Dense(outcomes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])
    return model
model = getModel()
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 200)               2000      
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 125)               25125     
_________________________________________________________________
dense_2 (Dense)              (None, 75)                9450      
_________________________________________________________________
dropout_1 (Dropout)          (None, 75)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 25)                1900      
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 7

In [11]:
# Get a set of board states labelled by who eventually won that game
def gamesToWinLossData(games):
    X = []
    y = []
    for game in games:
        winner = getWinner(movesToBoard(game))
        for move in range(len(game)):
            X.append(movesToBoard(game[:(move + 1)]))
            y.append(winner)

    X = np.array(X).reshape((-1, 9))
    y = to_categorical(y)
    
    # Return an appropriate train/test split
    trainNum = int(len(X) * 0.8)
    return (X[:trainNum], X[trainNum:], y[:trainNum], y[trainNum:])

# Split out train and validation data
X_train, X_test, y_train, y_test = gamesToWinLossData(games)

In [12]:
nEpochs = 100
batchSize = 100
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=nEpochs, batch_size=batchSize)

Epoch 1/100
625/625 [==============================] - 1s 2ms/step - loss: 1.0032 - acc: 0.5178 - val_loss: 0.9682 - val_acc: 0.5479
Epoch 2/100
625/625 [==============================] - 1s 2ms/step - loss: 0.9415 - acc: 0.5516 - val_loss: 0.9052 - val_acc: 0.5798
Epoch 3/100
625/625 [==============================] - 1s 2ms/step - loss: 0.9056 - acc: 0.5658 - val_loss: 0.8664 - val_acc: 0.6023
Epoch 4/100
625/625 [==============================] - 1s 2ms/step - loss: 0.8829 - acc: 0.5773 - val_loss: 0.8447 - val_acc: 0.6127
Epoch 5/100
625/625 [==============================] - 1s 2ms/step - loss: 0.8658 - acc: 0.5886 - val_loss: 0.8323 - val_acc: 0.6209
Epoch 6/100
625/625 [==============================] - 1s 2ms/step - loss: 0.8500 - acc: 0.5978 - val_loss: 0.8226 - val_acc: 0.6301
Epoch 7/100
625/625 [==============================] - 1s 2ms/step - loss: 0.8397 - acc: 0.6047 - val_loss: 0.8188 - val_acc: 0.6310
Epoch 8/100
625/625 [==============================] - 1s 2ms/step - 

In [13]:
games2 = [simulateGame(p1=model) for _ in range(1000)]

In [14]:
gameStats(games2)

Results for player 1:
Wins: 920 (92.0%)
Loss: 11 (1.1%)
Draw: 69 (6.9%)


In [18]:
# Save the entire model as a SavedModel.
#!mkdir -p saved_model
model.save('/content/drive/My Drive/TIC TAC TOE/saved_model/tictactoe_model')

INFO:tensorflow:Assets written to: /content/drive/My Drive/TIC TAC TOE/saved_model/tictactoe_model/assets
